### NLP (Natural Language Processing)-machine learning Classification  3( N-gram vectors)

Dans l'article précedent, nous avons décomposé le workflow de classification de texte en plusieurs étapes. pour répondre à deux questions clés:


* Quel algorithme ou modèle d'apprentissage devrions-nous utiliser?

* Comment préparer les données pour apprendre efficacement la relation entre le texte et l'étiquette?


Pour chaque étape, nous avons proposé une approche personnalisée basée sur les caractéristiques spécifique de votre jeu de données. En particulier, en utilisant le rapport du nombre d'échantillons au nombre de mots par échantillon, nous avons suggéré deux approche .

La réponse à la deuxième question dépend de la réponse à la première question, la façon dont nous pré-traitons les données à introduire dans un modèle dépendra du modèle que nous choisirons. Les modèles peuvent être classés en deux grandes catégories:

* ceux qui utilisent le texte comme une séquence des mots (modèles de séquence) 

* ceux qui ne voient que le texte comme des «sac de mots>>  (modèles à n-grammes). 

Lorsque la valeur de ce rapport est petite (<1500), les modèles  qui prennent les n-grammes en entrée  fonctionnent mieux ou au moins aussi bien que les modèles de séquence.
Lorsque la valeur de ce rapport est grande (> = 1500), utilisez un modèle de séquence . 

Dans cet article , nous travaillerons à la construction, et à l'évaluation des modèles à n-grammes,
Les modèles  qui prennent les n-grammes en entrée sont simples à définir et à comprendre, et ils prennent moins de temps de calcul que les modèles de séquence.

Certains des algorithmes de classification à n-grammes les plus largement utilisés sont la régression logistique,les perceptrons multicouches(MLP),  Naive Bayes, le plus proche voisin K et les méthodes d'arbre.... ensembles

## Étape 4: Créez, formez et évaluez votre modèle

À l' étape 3 , nous avons choisi d'utiliser un modèle à n-grammes ou un modèle de séquence.
Maintenant, il est temps d'écrire notre algorithme de classification et de le former.

#### Construire un modèle n-gram

Nous nous référons aux modèles qui traitent les tokens indépendamment (sans tenir compte de l'ordre des mots) en tant que modèles à n-grammes. Les perceptrons multicouches  (y compris la régression logistique ), Naive Bayes, le plus proche voisin K et les méthodes d'arbre.... , entrent tous dans cette catégorie

Découvrons chacun d'eux ensembles

On recupere tout d'abord les données, et de créer deux DataFrame Pandas, un pour l'apprentissage, l'autre pour la validation.

In [1]:
import sklearn.model_selection as sms
import pandas as pd

In [2]:
def split_dataset(input_path, nb_line, tauxValid):
    data_all = pd.read_csv(input_path,sep=",", nrows=nb_line)
    data_all = data_all.fillna("")
    data_train, data_valid = sms.train_test_split(data_all, test_size=tauxValid, random_state=47, shuffle=True)
    return data_train, data_valid

In [5]:
input_path = "data/cdiscount_train.csv.zip"
nb_line=50000 # part totale extraite du fichier initial ici déjà réduit
tauxValid = 0.2
data_train, data_valid = split_dataset(input_path, nb_line, tauxValid)
data_train.reset_index(inplace = True)
data_valid.reset_index(inplace = True)

N_train = data_train.shape[0]
N_valid = data_valid.shape[0]
print("Train set : %d elements, Validation set : %d elements" %(N_train, N_valid))

Train set : 40000 elements, Validation set : 10000 elements


Vectoriser en utilisant l'encodage tf-idf,
Sélectionnez uniquement les 20 000 premières fonctionnalités du vecteur de jetons en supprimant les jetons qui apparaissent moins de 2 fois et en utilisant f_classif pour calculer l'importance des fonctionnalités.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from spacy.lang.fr import French

nlp = French()

def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower() for word in mytokens if
                not word.is_punct and not word.like_num and word.text != 'n']

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words]

    # Remove accentuated char for any unicode symbol
    mytokens = [strip_accents_ascii(word) for word in mytokens]

    # return preprocessed list of tokens
    return mytokens

# Vectorization parameters
# Range (inclusive) of n-gram sizes for tokenizing text.
NGRAM_RANGE = (1, 2)

# Limit on the number of features. We use the top 20K features.
TOP_K = 20000

# Whether text should be split into word or character n-grams.
# One of 'word', 'char'.
TOKEN_MODE = 'word'

# Minimum document/corpus frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY = 2

def ngram_vectorize(train_texts, train_labels, val_texts):
    """Vectorizes texts as n-gram vectors.

    1 text = 1 tf-idf vector the length of vocabulary of unigrams + bigrams.

    # Arguments
        train_texts: list, training text strings.
        train_labels: np.ndarray, training labels.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val: vectorized training and validation texts
    """
    # Create keyword arguments to pass to the 'tf-idf' vectorizer.
    kwargs = {
            'ngram_range': NGRAM_RANGE,  # Use 1-grams + 2-grams.
            'dtype': 'int32',
            'strip_accents': 'unicode',
            'decode_error': 'replace',
            'analyzer': TOKEN_MODE,  # Split text into word tokens.
            'min_df': MIN_DOCUMENT_FREQUENCY,
    }
    vectorizer = TfidfVectorizer(**kwargs)

    # Learn vocabulary from training texts and vectorize training texts.
    x_train = vectorizer.fit_transform(train_texts)

    # Vectorize validation texts.
    x_val = vectorizer.transform(val_texts)

    # Select top 'k' of the vectorized features.
    selector = SelectKBest(f_classif, k=min(TOP_K, x_train.shape[1]))
    selector.fit(x_train, train_labels)
    x_train = selector.transform(x_train).astype('float32')
    x_val = selector.transform(x_val).astype('float32')
    return x_train, x_val

In [7]:
x_train,x_val = ngram_vectorize(data_train["Description"], data_train["Categorie1"], data_valid["Description"])

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


###### Bagging

Le principe des méthodes de Bagging, et donc en particulier des forêts aléatoires c’est de faire la moyenne des prévisions de plusieurs modèles indépendants pour réduire la variance et donc l’erreur de prévision. Pour construire ces différents modèles, on sélectionne plusieurs échantillons bootstrap, c’est à dire des tirages avec remises.

![](data/bagging.png)

In [8]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics


y_train = data_train["Categorie1"]

y_test = data_valid["Categorie1"]

bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=50, n_jobs=-1)
bag_clf.fit(x_train, y_train)

predicted = bag_clf.predict(x_val)

print(metrics.classification_report(y_test, predicted))

                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.57      0.23      0.33        98
                          ANIMALERIE - NEW       0.55      0.28      0.37        64
            ARME DE COMBAT - ARME DE SPORT       1.00      0.11      0.20         9
     ART DE LA TABLE - ARTICLES CULINAIRES       0.45      0.33      0.38       111
                      ARTICLES POUR FUMEUR       0.84      0.40      0.54        40
                         AUTO - MOTO (NEW)       0.77      0.77      0.77       442
                                 BAGAGERIE       0.81      0.81      0.81       215
                   BATEAU MOTEUR - VOILIER       0.00      0.00      0.00         6
              BIJOUX -  LUNETTES - MONTRES       0.87      0.94      0.90       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.31      0.54      0.39       274
                  CHAUSSURES - ACCESSOIRES       0.81      0.65      0.72  

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


######  Random Forest

Cet algorithme appartient à la famille des agrégations de modèles, c’est en fait un cas particulier de bagging (bootstrap aggregating) appliqué aux arbres de décision de type CART. 
En plus du principe de bagging, les forêts aléatoires ajoutent de l’aléa au niveau des variables. Pour chaque arbre on sélectionne un échantillon bootstrat d’individus et à chaque étape, la construction d’un noeud de l’arbre se fait sur un sous-ensemble de variables tirées aléatoirement.

On se retrouve donc avec plusieurs arbres et donc des prédictions différentes pour chaque individu. Comment obtenir l’estimation finale?

* Dans le cas d’une classification : on choisit la catégorie la plus fréquente
* Dans le cas d’une régression : on fait la moyenne des valeurs prédites

![](data/forest.png)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics


y_train = data_train["Categorie1"]
y_test = data_valid["Categorie1"]

RandFores_clf = RandomForestClassifier(n_estimators=50,class_weight="balanced",n_jobs=-1)
RandFores_clf.fit(x_train, y_train)

predicted = RandFores_clf.predict(x_val)
print(metrics.classification_report(y_test, predicted))

                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.69      0.34      0.45        98
                          ANIMALERIE - NEW       0.74      0.39      0.51        64
            ARME DE COMBAT - ARME DE SPORT       1.00      0.44      0.62         9
     ART DE LA TABLE - ARTICLES CULINAIRES       0.80      0.36      0.50       111
                      ARTICLES POUR FUMEUR       0.83      0.62      0.71        40
                         AUTO - MOTO (NEW)       0.83      0.79      0.81       442
                                 BAGAGERIE       0.82      0.82      0.82       215
                   BATEAU MOTEUR - VOILIER       0.00      0.00      0.00         6
              BIJOUX -  LUNETTES - MONTRES       0.89      0.94      0.92       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.43      0.53      0.48       274
                  CHAUSSURES - ACCESSOIRES       0.88      0.75      0.81  

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


###### Boosting (AdaBoost)

L’idée de base ressemble à celle du bagging. Plutôt que d’utiliser un seul modèle, nous en utilisons plusieurs que nous agrégeons ensuite pour obtenir un seul résultat. Dans  la construction des modèles, le Boosting travaille de manière séquentielle. Il commence par construire un premier modèle qu’il va évaluer. A partir de cette mesure, chaque individu va être pondéré en fonction de la performance de la prédiction. L’objectif est de donner un poids plus important aux individus pour lesquels la valeur a été mal prédite pour la construction du modèle suivant. Le fait de corriger les poids au fur et à mesure permet de mieux prédire les valeurs difficiles.



<img src="data/ada.png">

In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

y_train = data_train["Categorie1"]
y_test = data_valid["Categorie1"]

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=40,algorithm="SAMME.R",learning_rate=0.3)
ada_clf.fit(x_train, y_train)

predicted = ada_clf.predict(x_val)
print(metrics.classification_report(y_test, predicted))

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.44      0.24      0.32        98
                          ANIMALERIE - NEW       0.47      0.34      0.40        64
            ARME DE COMBAT - ARME DE SPORT       0.33      0.11      0.17         9
     ART DE LA TABLE - ARTICLES CULINAIRES       0.41      0.21      0.28       111
                      ARTICLES POUR FUMEUR       0.81      0.42      0.56        40
                         AUTO - MOTO (NEW)       0.75      0.76      0.75       442
                                 BAGAGERIE       0.81      0.78      0.80       215
                   BATEAU MOTEUR - VOILIER       1.00      0.17      0.29         6
              BIJOUX -  LUNETTES - MONTRES       0.89      0.93      0.91       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.31      0.49      0.38       274
                  CHAUSSURES - ACCESSOIRES       0.84      0.60      0.70  

###### gradient boosting

L’algorithme de boosting de gradient (gradient boosting) figure aussi parmi les méthodes de boosting très appréciées. Tout comme AdaBoost, le boosting de gradient travaille par ajout séquentiel de prédicteurs à un ensemble, chacun d’eux corrigeant son prédécesseur. Cependant, au lieu de modier légèrement les poids des observations à chaque itération comme le fait AdaBoost, cette méthode tente d’ajuster un nouveau prédicteur aux erreurs résiduelles du prédicteur précédent.

Cet algorithme utilise le gradient de la fonction de perte pour le calcul des poids des individus lors de la construction de chaque nouveau modèle. Cela ressemble un peu à la descente de gradient pour les réseaux de neurones.

<img src="data/Gradient_Boosting.png">

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

y_train = data_train["Categorie1"]
y_test = data_valid["Categorie1"]

GradientBoostin_clf = GradientBoostingClassifier(n_estimators=40)
GradientBoostin_clf.fit(x_train, y_train)

predicted = GradientBoostin_clf.predict(x_val)
print(metrics.classification_report(y_test, predicted))

                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.57      0.48      0.52        98
                          ANIMALERIE - NEW       0.53      0.33      0.40        64
            ARME DE COMBAT - ARME DE SPORT       0.38      0.33      0.35         9
     ART DE LA TABLE - ARTICLES CULINAIRES       0.61      0.48      0.54       111
                      ARTICLES POUR FUMEUR       0.89      0.78      0.83        40
                         AUTO - MOTO (NEW)       0.90      0.81      0.85       442
                                 BAGAGERIE       0.86      0.83      0.84       215
                   BATEAU MOTEUR - VOILIER       0.10      0.17      0.12         6
              BIJOUX -  LUNETTES - MONTRES       0.94      0.92      0.93       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.63      0.48      0.55       274
                  CHAUSSURES - ACCESSOIRES       0.89      0.84      0.87  

###### régression logistique
La régression logistique est utilisée couramment pour estimer la proba-bilité qu’une observation appartienne à une classe particulière (p. ex. quelle est la probabilité que cet e-mail soit un pourriel?). Si la probabilité estimée est supérieure à 50%, alors le modèle prédit que l’observation appartient à cette classe (appelée la classe positive, d’étiquette «1»), sinon il prédit qu’elle appartient à l’autre classe (la classe négative, d’étiquette «0»). C’est en fait un classificateur binaire.

Tout comme un modèle de régression linéaire, un modèle de régression logistique calcule une somme pondérée des caractéristiques d’entrée (plus un terme constant), mais au lieu de fournir le résultat directement comme le fait le modèle de régression linéaire, il fournit la logistique du résultat.

Le modèle de régression logistique peut être généralisé de manière à prendre en compte plusieurs classes directement, sans avoir à entraîner plusieurs classificateurs binaires puis à les combiner. C’est ce qu’on appelle la régression softmax, ou régression logistique multinomiale.
Le principe en est simple: étant donné une observation x, le modèle de régression softmax calcule d’abord un score s_k(x) pour chaque classe k, puis estime la probabilité de chaque classe en appliquant aux scores la fonction softmax . La formule permettant de calculer s_k(x) devrait vous sembler familière, car c’est la même que pour calculer la prédiction en régression linéaire.

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

y_train = data_train["Categorie1"]
y_test = data_valid["Categorie1"]

softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10, class_weight="balanced")
softmax_reg.fit(x_train, y_train)


predicted = softmax_reg.predict(x_val)
print(metrics.classification_report(y_test, predicted))

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.57      0.72      0.64        98
                          ANIMALERIE - NEW       0.55      0.48      0.52        64
            ARME DE COMBAT - ARME DE SPORT       0.75      0.33      0.46         9
     ART DE LA TABLE - ARTICLES CULINAIRES       0.59      0.74      0.66       111
                      ARTICLES POUR FUMEUR       0.94      0.85      0.89        40
                         AUTO - MOTO (NEW)       0.91      0.87      0.89       442
                                 BAGAGERIE       0.89      0.94      0.92       215
                   BATEAU MOTEUR - VOILIER       1.00      0.33      0.50         6
              BIJOUX -  LUNETTES - MONTRES       0.96      0.96      0.96       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.52      0.77      0.62       274
                  CHAUSSURES - ACCESSOIRES       0.89      0.93      0.91  

#### Machines à vecteurs  de support

Une machine à vecteurs de support ou séparateur à vaste marge (SVM) est un modèle d’apprentissage automatique à la fois puissant et polyvalent, capable d’effectuer des classifications linéaires ou non linéaires, des régressions et même des détections de données aberrantes. C’est un des modèles les plus prisés en matière d’apprentissage automatique, et tous ceux qui s’intéressent au Machine Learning devraient en avoir dans leur boîte à outils. Ils sont particulièrement bien adaptés à la classification de jeux de données complexes, de taille réduite ou moyenne.
L'idée de Support Vector Machines est simple: l'algorithme crée une ligne qui sépare les classes, dans le cas par exemple d'un problème de classification. Le but de la ligne est de maximiser la marge entre les points de chaque côté de la ligne dite de décision. L'avantage de ce processus est qu'après la séparation, le modèle peut facilement deviner les classes cibles (étiquettes) pour les nouveaux cas

<img src="data/svm.png">

In [25]:
from sklearn.svm import LinearSVC
from sklearn import metrics

y_train = data_train["Categorie1"]
y_test = data_valid["Categorie1"]

svm_clf = LinearSVC(class_weight="balanced")
svm_clf.fit(x_train, y_train)


predicted = svm_clf.predict(x_val)
print(metrics.classification_report(y_test, predicted))

                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.59      0.74      0.66        98
                          ANIMALERIE - NEW       0.46      0.48      0.47        64
            ARME DE COMBAT - ARME DE SPORT       0.62      0.56      0.59         9
     ART DE LA TABLE - ARTICLES CULINAIRES       0.60      0.73      0.66       111
                      ARTICLES POUR FUMEUR       0.90      0.90      0.90        40
                         AUTO - MOTO (NEW)       0.92      0.88      0.90       442
                                 BAGAGERIE       0.87      0.94      0.90       215
                   BATEAU MOTEUR - VOILIER       0.50      0.33      0.40         6
              BIJOUX -  LUNETTES - MONTRES       0.96      0.97      0.97       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.64      0.72      0.68       274
                  CHAUSSURES - ACCESSOIRES       0.89      0.94      0.92  

###### Naive Bayes Classifier
La classification bayésienne naïve (Naive Bayes classifier), est utilisée depuis longtemps dans l'industrie et le monde universitaire (introduite par Thomas Bayes ), Cependant, cette technique est à l'étude depuis les années 50 pour la catégorisation des textes et des documents. 
Naive Bayes Classifier est un modèle génératif qui est largement utilisé dans la recherche d'informations, qui se base sur le théorème de Bayes. 

<img src="data/NB.png">

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

y_train = data_train["Categorie1"]
y_test = data_valid["Categorie1"]

NB_clf = MultinomialNB()
NB_clf.fit(x_train, y_train)


predicted = NB_clf.predict(x_val)
print(metrics.classification_report(y_test, predicted))

                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.00      0.00      0.00        98
                          ANIMALERIE - NEW       1.00      0.03      0.06        64
            ARME DE COMBAT - ARME DE SPORT       0.00      0.00      0.00         9
     ART DE LA TABLE - ARTICLES CULINAIRES       1.00      0.04      0.07       111
                      ARTICLES POUR FUMEUR       1.00      0.10      0.18        40
                         AUTO - MOTO (NEW)       0.90      0.81      0.85       442
                                 BAGAGERIE       0.96      0.52      0.67       215
                   BATEAU MOTEUR - VOILIER       0.00      0.00      0.00         6
              BIJOUX -  LUNETTES - MONTRES       0.76      0.94      0.84       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.78      0.34      0.47       274
                  CHAUSSURES - ACCESSOIRES       0.94      0.40      0.56  

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


###### réseaux de neurones profond

Les architectures des réseaux neuronaux profonds sont conçues pour apprendre à travers la connexion multiple de couches où chaque couche unique ne reçoit que la connexion de la précédente et fournit des connexions uniquement à la couche suivante dans la partie cachée. L'entrée est une connexion de l'espace d'entité (avec la première couche cachée). Pour les réseaux neuronaux profonds , la couche d'entrée peut être tf-ifd, one-hot encoding etc.., la couche de sortie contient des neurones égaux au nombre de classes pour la classification multi-classes, et un seul neurone pour la classification binaire.

Lorsque nous n'avons que 2 classes (classification binaire), notre modèle devrait produire un seul score de probabilité. Par exemple, la sortie 0.2 pour un échantillon d'entrée donné signifie «20% de confiance que cet échantillon est dans la première classe (classe 1), 80% qu'il est dans la deuxième classe (classe 0)». Pour produire un tel score de probabilité, la fonction d'activation de la dernière couche doit être une fonction sigmoïde et la fonction de perte utilisée pour entraîner le modèle doit être une entropie croisée binaire (binary cross-entropy) .

Lorsqu'il y a plus de 2 classes (classification multi-classes), notre modèle devrait produire un score de probabilité par classe. La somme de ces scores doit être 1. Par exemple, la sortie {0: 0.2, 1: 0.7, 2: 0.1}signifie «20% de confiance que cet échantillon est dans la classe 0, 70% qu'il est dans la classe 1 et 10% qu'il est dans la classe 2.» Pour produire ces scores, la fonction d'activation de la dernière couche doit être softmax et la fonction de perte utilisée pour entraîner le modèle doit être une entropie croisée catégorique(categorical cross-entropy).

<img src="data/RN.jpg">

In [60]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

y_train = data_train[["Categorie1"]]
y_test = data_valid[["Categorie1"]]


cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(y_train) 

ordinal_encoder = OrdinalEncoder()
train_labels = ordinal_encoder.fit_transform(y_train)
val_labels = ordinal_encoder.transform(y_train)






(max(val_labels)+1)[0]

43.0

In [64]:
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout


def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    # Arguments
        num_classes: int, number of classes.

    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    
    return model

In [88]:
# Get the data.
from sklearn.preprocessing import OrdinalEncoder
data = ((data_train["Description"], data_train[["Categorie1"]]), (data_valid["Description"], data_valid[["Categorie1"]]))
(train_texts, train_labels), (val_texts, val_labels) = data


# Vectorize texts.
x_train, x_val = ngram_vectorize(
        train_texts, train_labels, val_texts)
    
ordinal_encoder = OrdinalEncoder()
train_labels = ordinal_encoder.fit_transform(train_labels)
val_labels = ordinal_encoder.transform(val_labels)
    
num_classes = (max(val_labels)+1)[0]

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)
C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [89]:
import tensorflow as tf


def train_ngram_model(learning_rate=1e-3,
                      epochs=1000,
                      batch_size=128,
                      layers=2,
                      units=64,
                      dropout_rate=0.2):
    """Trains n-gram model on the given dataset.

    # Arguments
        data: tuples of training and test texts and labels.
        learning_rate: float, learning rate for training model.
        epochs: int, number of epochs.
        batch_size: int, number of samples per batch.
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of Dense layers in the model.
        dropout_rate: float: percentage of input to drop at Dropout layers.

    # Raises
        ValueError: If validation data has label values which were not seen
            in the training data.
    """    
    # Create model instance.
    model = mlp_model(layers=layers,
                                  units=units,
                                  dropout_rate=dropout_rate,
                                  input_shape=x_train.shape[1:],
                                  num_classes=num_classes)
    

    # Compile model with learning parameters.
    if num_classes == 2:
        loss = 'binary_crossentropy'
    else:
        loss = 'sparse_categorical_crossentropy'
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

    # Create callback for early stopping on validation loss. If the loss does
    # not decrease in two consecutive tries, stop training.
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=2)]

    # Train and validate model.
    history = model.fit(
            x_train,
            train_labels,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=(x_val, val_labels),
            verbose=2,  # Logs once per epoch.
            batch_size=batch_size)

    # Print results.
    history = history.history
    print('Validation accuracy: {acc}, loss: {loss}'.format(
            acc=history['val_acc'][-1], loss=history['val_loss'][-1]))

    # Save model.
    #model.save('IMDb_mlp_model.h5')
    return model

In [90]:
model= train_ngram_model()

Train on 40000 samples, validate on 10000 samples
Epoch 1/1000
40000/40000 - 17s - loss: 2.2191 - acc: 0.5762 - val_loss: 1.3085 - val_acc: 0.7256
Epoch 2/1000
40000/40000 - 18s - loss: 1.1026 - acc: 0.7596 - val_loss: 0.8862 - val_acc: 0.8056
Epoch 3/1000
40000/40000 - 18s - loss: 0.7858 - acc: 0.8231 - val_loss: 0.6958 - val_acc: 0.8431
Epoch 4/1000
40000/40000 - 17s - loss: 0.6075 - acc: 0.8602 - val_loss: 0.5920 - val_acc: 0.8611
Epoch 5/1000
40000/40000 - 17s - loss: 0.4892 - acc: 0.8862 - val_loss: 0.5277 - val_acc: 0.8747
Epoch 6/1000
40000/40000 - 17s - loss: 0.4089 - acc: 0.9048 - val_loss: 0.4881 - val_acc: 0.8801
Epoch 7/1000
40000/40000 - 17s - loss: 0.3509 - acc: 0.9169 - val_loss: 0.4651 - val_acc: 0.8840
Epoch 8/1000
40000/40000 - 17s - loss: 0.3105 - acc: 0.9259 - val_loss: 0.4516 - val_acc: 0.8866
Epoch 9/1000
40000/40000 - 17s - loss: 0.2733 - acc: 0.9349 - val_loss: 0.4428 - val_acc: 0.8873
Epoch 10/1000
40000/40000 - 17s - loss: 0.2467 - acc: 0.9386 - val_loss: 0.43

In [113]:
import pandas as pd  
predicted = model.predict_classes(x_val)
predicted = pd.DataFrame(predicted)
predicted = ordinal_encoder.inverse_transform(e_dataframe)
print(metrics.classification_report(data_valid["Categorie1"], predicted))

                                            precision    recall  f1-score   support

                        ADULTE - EROTIQUE        0.84      0.64      0.73        98
                          ANIMALERIE - NEW       0.81      0.45      0.58        64
            ARME DE COMBAT - ARME DE SPORT       1.00      0.11      0.20         9
     ART DE LA TABLE - ARTICLES CULINAIRES       0.68      0.70      0.69       111
                      ARTICLES POUR FUMEUR       1.00      0.72      0.84        40
                         AUTO - MOTO (NEW)       0.89      0.90      0.89       442
                                 BAGAGERIE       0.90      0.92      0.91       215
                   BATEAU MOTEUR - VOILIER       0.00      0.00      0.00         6
              BIJOUX -  LUNETTES - MONTRES       0.94      0.98      0.96       591
     BRICOLAGE - OUTILLAGE - QUINCAILLERIE       0.60      0.74      0.66       274
                  CHAUSSURES - ACCESSOIRES       0.90      0.91      0.91  

C:\Users\rachi\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Étape 5:régler les hyperparamètres

Nous avons dû choisir un certain nombre d'hyperparamètres pour définir et former le modèle. Nous nous sommes appuyés sur l'intuition, des exemples et des recommandations de bonnes pratiques. Cependant, notre premier choix de valeurs hyperparamétriques peut ne pas donner les meilleurs résultats. Cela ne nous donne qu'un bon point de départ pour la formation. Chaque problème est différent et le réglage de ces hyperparamètres aidera à affiner votre modèle. pour mieux représenter les particularités du problème à résoudre.

In [ ]:
https://lovelyanalytics.com/
https://spacy.io/
https://developers.google.com/
https://github.com/kk7nc/Text_Classification
https://www.tensorflow.org/
https://scikit-learn.org/stable/
https://medium.com/@LSchultebraucks/introduction-to-support-vector-machines-9f8161ae2fcb